<a href="https://colab.research.google.com/github/devShaurya/Summer-Intern/blob/master/Cubicasa5kFinal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Initial Download and Prepare Dataset

### Download the Dataset

In [ ]:
!wget -N https://zenodo.org/record/2613548/files/cubicasa5k.zip?download=1 -O cubicasa5k.zip

In [ ]:
!unzip cubicasa5k.zip -d cubicasa5k/

In [ ]:
%cd cubicasa5k

I renamed the folder here inside the top cubicasa5k directory to "cubicasa5kData"

In [ ]:
!git clone https://github.com/CubiCasa/CubiCasa5k CubiCasa5kCode  ## clone the code of cubicasa5k to CubiCasa5kCode directory inside cubicasa5k 

In [ ]:
!mkdir cubicasa5k_masks

### Change the directory to the directory where code is saved

In [ ]:
%cd "/content/drive/My Drive/Accenture Summer Intern/cubicasa5k/CubiCasa5kCode"

/content/drive/My Drive/Accenture Summer Intern/cubicasa5k/CubiCasa5kCode


### Install and import libraries

In [ ]:
!pip install svgpathtools

     |████████████████████████████████| 51kB 928kB/s 
     |████████████████████████████████| 71kB 2.2MB/s 


In [ ]:
import lmdb
import os
import pickle
import argparse
import logging
import numpy as np
from datetime import datetime
from tqdm import tqdm
from floortrans.loaders.svg_loader import FloorplanSVG
import cv2
import matplotlib.pyplot as plt
from PIL import Image
import shutil
import random

### Save masks

In [ ]:
def saveMasksCubicasa(textFilePath):
    data = FloorplanSVG("/content/drive/My Drive/Accenture Summer Intern/cubicasa5k/cubicasa5kData/",textFilePath, format='txt', original_size=True)    
    folders = np.genfromtxt("/content/drive/My Drive/Accenture Summer Intern/cubicasa5k/cubicasa5kData/{}".format(textFilePath), dtype='str')
    path = "/content/drive/My Drive/Accenture Summer Intern/cubicasa5k/cubicasa5k"
    total=0
    for i, f in tqdm(enumerate(folders), total=len(folders)):
        arr=f.strip('/').split('/')
        imPath=path+'_masks/'+arr[0]+'_masks/'+arr[-1]+'/'
        if len(os.listdir(path+'/'+arr[0]))==0:
            continue
        if not os.path.exists(imPath):
            os.mkdir(imPath)
        try:
            label=np.array(data[i]["label"])
            floorMask=label[0,:,:]
            iconMask=label[1,:,:]
            cv2.imwrite(imPath+"room_mask.bmp",floorMask)
            cv2.imwrite(imPath+"icon_mask.bmp",iconMask)
            total+=1
        except:
            continue
    return total

In [ ]:
# textFilePath="test.txt"
# test=saveMasksCubicasa(textFilePath)
# textFilePath="val.txt"
# val=saveMasksCubicasa(textFilePath)
# textFilePath="train.txt"
# train=saveMasksCubicasa(textFilePath)

###Save walls

In [ ]:
def wallGT(path):
    path=path.rstrip('/')
    folders=os.listdir(path)
    for ind,folder in tqdm(enumerate(folders),total=len(folders)):
        try:
            roomMaskPath=path+'/'+folder+'/'+"room_mask.bmp"
            roomMask=cv2.imread(roomMaskPath)
            roomMask=roomMask[:,:,0]
            roomMask[roomMask!=2]=255
            roomMask[roomMask!=255]=0
            wallMask=path+'/'+folder+'/'+'wall_mask.bmp'
            cv2.imwrite(wallMask,roomMask)
        # return folder
        except:
            continue
        # break
    print("completed")

In [ ]:
# room=wallGT("/content/drive/My Drive/Accenture Summer Intern/cubicasa5k/cubicasa5k_masks/colorful_masks")
# room=wallGT("/content/drive/My Drive/Accenture Summer Intern/cubicasa5k/cubicasa5k_masks/high_quality_masks")
# room=wallGT("/content/drive/My Drive/Accenture Summer Intern/cubicasa5k/cubicasa5k_masks/high_quality_architectural_masks")

### Remove Empty Folders
1. colorful had 56 empty

In [ ]:
def removeEmpty(typed):
    path="/content/drive/My Drive/Accenture Summer Intern/cubicasa5k/cubicasa5k_masks/{}_masks".format(typed)    
    folders=[folder for folder in os.listdir(path)]
    origPath="/content/drive/My Drive/Accenture Summer Intern/cubicasa5k/cubicasa5kData/{}".format(typed)
    delDir=[]
    for folder in folders:
        # print(folder)
        if len(os.listdir('{}/{}'.format(path,folder)))==0 and len(os.listdir('{}/{}'.format(origPath,folder)))==0:
            delDir.append(folder)
            os.rmdir("{}/{}".format(path,folder))
            # break
    return len(delDir)

In [ ]:
x1=removeEmpty("high_quality_architectural")
x2=removeEmpty("high_quality")
x3=removeEmpty("colorful")

### Remove with more than one floor plan


1.   high_quality had 204 such plans

2.    high_quality_architectural had such 754 plans

3.   color had 53 such plans

In [ ]:
def giveFolders(typed):
    approved=["model.svg","F1_scaled.png","F1_original.png"]
    path="/content/drive/My Drive/Accenture Summer Intern/cubicasa5k/cubicasa5kData/{}/".format(typed)
    maskPath="/content/drive/My Drive/Accenture Summer Intern/cubicasa5k/cubicasa5k_masks/{}_masks/".format(typed)
    folders=[folder+'/' for folder in os.listdir(path)]
    toDel=[]
    for folder in folders:
        files=os.listdir(path+folder)
        for file in files:
            if(file not in approved):
                toDel.append(folder)
                if os.path.exists(maskPath+folder):
                    shutil.rmtree(maskPath+folder)
                break
    return toDel

In [ ]:
x1=giveFolders("colorful")
x2=giveFolders("high_quality")
x3=giveFolders("high_quality_architectural")

### Mixing labels (outside and railing)

In [ ]:
def mixRailandOut(typed):
    path="/content/drive/My Drive/Accenture Summer Intern/cubicasa5k/cubicasa5k_masks/{}_masks/".format(typed)
    folders=[folder+'/' for folder in os.listdir(path)]
    for folder in folders:
        roomMask=np.array(Image.open(path+folder+'room_mask.bmp'))
        iconMask=np.array(Image.open(path+folder+'icon_mask.bmp'))
        roomMask[roomMask==8]=2
        roomMask[roomMask==1]=11
        Image.fromarray(roomMask).save(path+folder+'new_room_mask.bmp')

In [ ]:
mixRailandOut('colorful')

In [ ]:
mixRailandOut('high_quality')

In [ ]:
mixRailandOut('high_quality_architectural')

### Combine Door and Window

In [ ]:
def combineDoorAndWind(typed):
    path="/content/drive/My Drive/Accenture Summer Intern/cubicasa5k/cubicasa5k_masks/{}_masks/".format(typed)
    folders=[folder+'/' for folder in os.listdir(path)]
    ### window 1 -> 12
    ### door 2 -> 13
    for folder in tqdm(folders):
        roomMask=np.array(Image.open(path+folder+'new_room_mask.bmp'))
        iconMask=np.array(Image.open(path+folder+'icon_mask.bmp'))
        roomMask[iconMask==1]=12
        roomMask[iconMask==2]=13
        Image.fromarray(roomMask).save(path+folder+"new_room_door_wind.bmp")

In [ ]:
combineDoorAndWind('colorful')
combineDoorAndWind('high_quality_architectural')
combineDoorAndWind('high_quality')

### Combine Furniture Labels

In [ ]:
def combineFurnIcons(typed):
    path="/content/drive/My Drive/Accenture Summer Intern/cubicasa5k/cubicasa5k_masks/{}_masks/".format(typed)
    folders=[folder+'/' for folder in os.listdir(path)]
    ### closet:3 -> 14, 
    ### sink:6 -> 15, 
    ### bathtub:9 -> 16,
    ### toilet:5 -> 17

    for folder in folders:
        roomMask=np.array(Image.open(path+folder+'new_room_door_wind.bmp'))
        iconMask=np.array(Image.open(path+folder+'icon_mask.bmp'))
        roomMask[iconMask==3]=14
        roomMask[iconMask==6]=15
        roomMask[iconMask==9]=16
        roomMask[iconMask==5]=17
        Image.fromarray(roomMask).save(path+folder+"new_room_all.bmp")

In [ ]:
combineFurnIcons('colorful')
combineFurnIcons('high_quality_architectural')
combineFurnIcons('high_quality')

### Save New Wall masks


In [ ]:
def getWall(image):
    image=np.array(image)
    newImage=np.zeros(image.shape)
    newImage[newImage==0]=255
    newImage[image==2]=0
    return newImage

In [ ]:
def mainWall(typed):
    path="/content/drive/My Drive/Accenture Summer Intern/cubicasa5k/cubicasa5k_masks/{}_masks/".format(typed)
    folders=[folder+'/' for folder in os.listdir(path)]
    for folder in tqdm(folders):
        roomMask=Image.open(path+folder+'new_room_mask.bmp')
        wallMask=getWall(roomMask)
        Image.fromarray(wallMask.astype(np.uint8)).save(path+folder+"new_wall_mask.bmp")

In [ ]:
mainWall('colorful')

100%|██████████| 168/168 [04:57<00:00,  1.77s/it]


[]

In [ ]:
mainWall('high_quality')

100%|██████████| 788/788 [31:58<00:00,  2.43s/it]


[]

In [ ]:
mainWall('high_quality_architectural')

100%|██████████| 2978/2978 [2:21:56<00:00,  2.86s/it]


[]

### Save Boundary Masks

In [ ]:
def getBoundary(image):
    wallImage=np.array(image).astype(np.uint8)
    wallImage=cv2.bitwise_not(wallImage)
    contour,hierarchy=cv2.findContours(wallImage,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
    plain=np.zeros((wallImage.shape[0],wallImage.shape[1],3),dtype=np.uint8)
    return cv2.bitwise_not(cv2.drawContours(plain,contour,0,(255,255,255)))

In [ ]:
def mainBoundary(typed):
    path="/content/drive/My Drive/Accenture Summer Intern/cubicasa5k/cubicasa5k_masks/{}_masks/".format(typed)
    folders=[folder+'/' for folder in os.listdir(path)]
    for folder in tqdm(folders):
        wallMask=Image.open(path+folder+'new_wall_mask.bmp')
        boundary=getBoundary(wallMask)
        Image.fromarray(boundary.astype(np.uint8)).convert('L').save(path+folder+"new_boundary_mask.bmp")

In [ ]:
mainBoundary('colorful')

In [ ]:
mainBoundary('high_quality')

In [ ]:
mainBoundary('high_quality_architectural')

100%|██████████| 2978/2978 [54:50<00:00,  1.10s/it]


### Save corners (Not implemented Completely)


In [ ]:
def saveCornersCubicasa(textFilePath):
    data = FloorplanSVG("/content/drive/My Drive/Accenture Summer Intern/cubicasa5k/cubicasa5kData/",textFilePath, format='txt', original_size=True)    
    folders = np.genfromtxt("/content/drive/My Drive/Accenture Summer Intern/cubicasa5k/cubicasa5kData/{}".format(textFilePath), dtype='str')
    path = "/content/drive/My Drive/Accenture Summer Intern/cubicasa5k/cubicasa5k"
    total=0
    print(data)
    for i in range(len(folders)):
        print(data[i]['label'])
        break
    # for i, f in tqdm(enumerate(folders), total=len(folders)):
    #     arr=f.strip('/').split('/')
    #     imPath=path+'_masks/'+arr[0]+'_masks/'+arr[-1]+'/'
    #     if len(os.listdir(path+'/'+arr[0]))==0:
    #         continue
    #     if not os.path.exists(imPath):
    #         os.mkdir(imPath)
    #     try:
    #         label=np.array(data[i]["label"])
    #         floorMask=label[0,:,:]
    #         iconMask=label[1,:,:]
    #         cv2.imwrite(imPath+"room_mask.bmp",floorMask)
    #         cv2.imwrite(imPath+"icon_mask.bmp",iconMask)
    #         total+=1
    #     except:
    #         continue
    # return total

In [ ]:
saveCornersCubicasa("train.txt")

tensor([[[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]]])


## Create A and B mapping for pix2pix

### Follow the notebook in "pix2pix repository". It has instructions for creating AtoB mapping which is required for pix2pix code.

Below code could be used to put the files in A directory. Just change paths according to yourselves

In [ ]:
def putFilesInA():
    pathColorful="/content/drive/My Drive/Accenture Summer Intern/cubicasa5k/cubicasa5k_masks/{}_masks/".format('colorful')
    pathHigh="/content/drive/My Drive/Accenture Summer Intern/cubicasa5k/cubicasa5k_masks/{}_masks/".format('high_quality')
    pathHighArch="/content/drive/My Drive/Accenture Summer Intern/cubicasa5k/cubicasa5k_masks/{}_masks/".format('high_quality_architectural')
    pathA="/content/drive/My Drive/Accenture Summer Intern/cubicasa5k/pix2pixv2/A/"
    foldersx=[pathColorful+folder+'/' for folder in os.listdir(pathColorful)]
    foldersy=[pathHigh+folder+'/' for folder in os.listdir(pathHigh)]
    foldersz=[pathHighArch+folder+'/' for folder in os.listdir(pathHighArch)]
    folders=foldersx+foldersy+foldersz
    folders.sort()
    i=1
    for folder in tqdm(folders):
        shutil.copy(folder+"new_boundary_mask.bmp",pathA+str(i)+'.bmp')
        i+=1

Below code could be used to put the files in B directory. Just change paths according to yourselves. Change the color map "d" according to yourselves

In [16]:
"""
0: ['Background'],
2: ['Wall','Railing'] 
3: ['Kitchen'],
4: ['Dining', 'EatingArea', 'LivingRoom', 'Lounge'],
5: ['Bedroom'],
6: ['Bath', 'Sauna'],
7: ['DraughtLobby', 'Entry', 'HallWay'],
9: ['Closet', 'DressingRoom', 'Storage'],
10: ['CarPort', 'Garage'],
11: [Other Rooms, 'Outside'],
12:['Window'],
13:['Door']
"""
def putFilesInB():
    pathColorful="/content/drive/My Drive/Accenture Summer Intern/cubicasa5k/cubicasa5k_masks/{}_masks/".format('colorful')
    pathHigh="/content/drive/My Drive/Accenture Summer Intern/cubicasa5k/cubicasa5k_masks/{}_masks/".format('high_quality')
    pathHighArch="/content/drive/My Drive/Accenture Summer Intern/cubicasa5k/cubicasa5k_masks/{}_masks/".format('high_quality_architectural')
    pathB="/content/drive/My Drive/Accenture Summer Intern/cubicasa5k/pix2pixv2/B/"
    foldersx=[pathColorful+folder+'/' for folder in os.listdir(pathColorful)]
    foldersy=[pathHigh+folder+'/' for folder in os.listdir(pathHigh)]
    foldersz=[pathHighArch+folder+'/' for folder in os.listdir(pathHighArch)]
    folders=foldersx+foldersy+foldersz
    folders.sort()
    i=1
    d={0:[0,255,255],2:[0,0,0],3:[128,0,128],4:[0,0,255],5:[255,0,0],6:[255,255,0],
       7:[0,100,0],9:[0,128,128], 
       10:[128,128,0],11:[255,165,0],12:[184,134,11],13:[160,82,45],
       14:[224,255,255],15:[205,133,63],16:[112,128,144],17:[138,43,226]}
    for folder in tqdm(folders):
        if(os.path.exists(pathB+str(i)+".bmp")):
            i+=1
            continue
        image=np.array(Image.open(folder+"new_room_mask.bmp"))
        newImage=np.zeros((image.shape[0],image.shape[1],3),dtype=np.uint8)
        for k in d:
            newImage[np.where(image==k)]=d[k]
        plt.imsave(pathB+str(i)+".bmp",newImage)
        i+=1


### make Train, test and val in A and B

Below code can be used to make test,train and val directories inside cubicasa folder. Change path according to yourselves

In [ ]:
os.mkdir("/content/drive/My Drive/Accenture Summer Intern/cubicasa5k/pix2pix/A/train")
os.mkdir("/content/drive/My Drive/Accenture Summer Intern/cubicasa5k/pix2pix/B/train")
os.mkdir("/content/drive/My Drive/Accenture Summer Intern/cubicasa5k/pix2pix/A/val")
os.mkdir("/content/drive/My Drive/Accenture Summer Intern/cubicasa5k/pix2pix/B/val")
os.mkdir("/content/drive/My Drive/Accenture Summer Intern/cubicasa5k/pix2pix/A/test")
os.mkdir("/content/drive/My Drive/Accenture Summer Intern/cubicasa5k/pix2pix/B/test")

Below functions shuffles and split the data 85-10-5 ratio. Change path and ratio acc to yourselves

In [ ]:
def moveFiles(subFolder,files):
    pathA="/content/drive/My Drive/Accenture Summer Intern/cubicasa5k/pix2pix/A/"
    pathB="/content/drive/My Drive/Accenture Summer Intern/cubicasa5k/pix2pix/B/"
    for i in tqdm(files):
        currA=pathA+i
        newA=pathA+subFolder+'/'+i
        shutil.move(currA,newA)
        currB=pathB+i
        newB=pathB+subFolder+'/'+i
        shutil.move(currB,newB)

In [ ]:
def makeTrainTestVal(trainSplit=0.85,valSplit=0.1):
    files=[str(i)+'.bmp' for i in range(1,3935)]
    random.shuffle(files)
    # trainSplit=0.85
    # valSplit=0.1
    # testSplit=0.05
    trainFiles=files[:int(len(files)*trainSplit)]##
    valFiles=files[int(len(files)*trainSplit):int(len(files)*(trainSplit+valSplit))]
    testFiles=files[int(len(files)*trainSplit+valSplit):]
    moveFiles('train',trainFiles)
    moveFiles('test',testFiles)
    moveFiles('val',valFiles)

In [ ]:
makeTrainTestVal()